In [1]:
!pip install tensorflow_datasets


You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:

import tensorflow as tf

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)


#used to fix bug in keras preprocessing scope
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
print("processed")

processed


In [9]:
IMAGE_SIZE = (224, 224)
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
BATCH_SIZE_PER_GPU = 96
global_batch_size = (BATCH_SIZE_PER_GPU * 1)
NUM_CLASSES = 10

Dataset code

In [10]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=IMAGE_SIZE)
        # Return a random crop
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random.uniform(())

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

def normalize(input_image):
  return preprocess_input(input_image)

@tf.function
def load_image_train(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  # if tf.random.uniform(()) > 0.5:
  #   input_image = tf.image.flip_left_right(input_image)
  augmentations = [flip, color, zoom, rotate]
  for f in augmentations:
    input_image = tf.cond(tf.random.uniform(()) > 0.75, lambda: f(input_image), lambda: input_image)

  #input_image = preprocess_input(input_image)
  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

@tf.function
def load_image_test(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  #input_image = preprocess_input(input_image)

  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

In [11]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(TRAIN_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(VALIDATION_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [12]:
def add_layers(inputs, layers=2):
    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=inputs.get_shape()[-1], 
                                        kernel_size= (3,3),
                                        padding='Same')(inputs)
    #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    
    build_replacement.counter += 1
    
    for i in range(1, layers):
        X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=inputs.get_shape()[-1],
                                            kernel_size=(3,3), 
                                            padding='Same')(X)
        #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
        X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
        build_replacement.counter += 1
    
    return X
    
def build_replacement(get_output, layers=2):
    inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
    
    X = add_layers(inputs, layers)
    replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)
    return replacement_layers

build_replacement.counter = 0

In [13]:

def make_list(X):
    if isinstance(X, list):
        return X
    return [X]

def list_no_list(X):
    if len(X) == 1:
        return X[0]
    return X

def replace_layer(model, replace_layer_subname, replacement_fn,
**kwargs):
    """
    args:
        model :: keras.models.Model instance
        replace_layer_subname :: str -- if str in layer name, replace it
        replacement_fn :: fn to call to replace all instances
            > fn output must produce shape as the replaced layers input
    returns:
        new model with replaced layers
    quick examples:
        want to just remove all layers with 'batch_norm' in the name:
            > new_model = replace_layer(model, 'batch_norm', lambda **kwargs : (lambda u:u))
        want to replace all Conv1D(N, m, padding='same') with an LSTM (lets say all have 'conv1d' in name)
            > new_model = replace_layer(model, 'conv1d', lambda layer, **kwargs: LSTM(units=layer.filters, return_sequences=True)
    """
    model_inputs = []
    model_outputs = []
    tsr_dict = {}

    model_output_names = [out.name for out in make_list(model.output)]

    for i, layer in enumerate(model.layers):
        ### Loop if layer is used multiple times
        for j in range(len(layer._inbound_nodes)):

            ### check layer inp/outp
            inpt_names = [inp.name for inp in make_list(layer.get_input_at(j))]
            outp_names = [out.name for out in make_list(layer.get_output_at(j))]

            ### setup model inputs
            if 'input' in layer.name:
                for inpt_tsr in make_list(layer.get_output_at(j)):
                    model_inputs.append(inpt_tsr)
                    tsr_dict[inpt_tsr.name] = inpt_tsr
                continue

            ### setup layer inputs
            # I added the exception model_3_3/Identity:0 I think the problem is that is the input layer
            inpt = list_no_list([tsr_dict[name]  for name in inpt_names])

            ### remake layer 
            if layer.name in replace_layer_subname:
              if "relu" in layer.name or 'bn' in layer.name:
                print('deleting ' + layer.name)
                x = inpt
              else:
                print('replacing '+layer.name)
                x = replacement_fn(inpt)
            else:
                x = layer(inpt)

            ### reinstantialize outputs into dict
            for name, out_tsr in zip(outp_names, make_list(x)):

                ### check if is an output
                if name in model_output_names:
                    model_outputs.append(out_tsr)
                tsr_dict[name] = out_tsr

    return tf.keras.models.Model(model_inputs, model_outputs)

In [14]:
def replac(inp):
    
    return add_layers(inp, layers=2)

In [15]:
dataset, info = tfds.load('cifar10', with_info=True)

make the upscaled cifar dataset

In [16]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train.shuffle(buffer_size=1000).batch(global_batch_size).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset['test'].map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(global_batch_size).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [17]:
model = tf.keras.models.load_model('./base_model_cifar10_resnet50.h5')

In [18]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=.01, momentum=.9, nesterov=True), loss='mse', metrics=['acc'])

In [19]:
results = model.evaluate(test_dataset, steps=VALIDATION_SIZE//global_batch_size)

  1/104 [..............................] - ETA: 6:29

InternalError:  Blas GEMM launch failed : a.shape=(96, 2048), b.shape=(2048, 10), m=96, n=10, k=2048
	 [[node model/dense/MatMul (defined at <ipython-input-19-2cb165b647e8>:1) ]] [Op:__inference_distributed_function_26548]

Function call stack:
distributed_function


## Make list of target layers to replace

In [15]:
import pprint
targets = []
for i, layer in enumerate(model.layers):
    if layer.__class__.__name__ == "Conv2D":
        if layer.kernel_size[0] == 3:
            #print(f'{i} layer {layer.name} , kernel size {layer.kernel_size}')
            targets.append({'name': layer.name, 'layer': i})

pprint.pprint(targets)

targets[0]['name'][:-4]

# add all layers to replace
for target in targets:
    for i, layer in enumerate(model.layers):
        if target['name'][:-4] in layer.name:
            if 'to_replace' in target:
                target['to_replace'].append((layer.name, i))
            else:
                target['to_replace'] = [(layer.name, i)]

[{'layer': 10, 'name': 'conv2_block1_2_conv'},
 {'layer': 22, 'name': 'conv2_block2_2_conv'},
 {'layer': 32, 'name': 'conv2_block3_2_conv'},
 {'layer': 42, 'name': 'conv3_block1_2_conv'},
 {'layer': 54, 'name': 'conv3_block2_2_conv'},
 {'layer': 64, 'name': 'conv3_block3_2_conv'},
 {'layer': 74, 'name': 'conv3_block4_2_conv'},
 {'layer': 84, 'name': 'conv4_block1_2_conv'},
 {'layer': 96, 'name': 'conv4_block2_2_conv'},
 {'layer': 106, 'name': 'conv4_block3_2_conv'},
 {'layer': 116, 'name': 'conv4_block4_2_conv'},
 {'layer': 126, 'name': 'conv4_block5_2_conv'},
 {'layer': 136, 'name': 'conv4_block6_2_conv'},
 {'layer': 146, 'name': 'conv5_block1_2_conv'},
 {'layer': 158, 'name': 'conv5_block2_2_conv'},
 {'layer': 168, 'name': 'conv5_block3_2_conv'}]


In [16]:
from hyperdash import monitor_cell

In [17]:
%%monitor_cell "layers"
for target in targets:
    
    print(f"training layer {target['name']}")
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model('base_model_cifar10_resnet50.h5')
    in_layer, out_layer = target['to_replace'][0][1], target['to_replace'][-1][1]
    get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[in_layer - 1].output, model.layers[out_layer].output])


    replacement_layers = build_replacement(get_output, layers=2)
    replacement_len = len(replacement_layers.layers)
    layer_train_gen = LayerBatch(get_output, train_dataset)
    layer_test_gen = LayerTest(get_output, test_dataset)




    MSE = tf.losses.MeanSquaredError()

    optimizer=tf.keras.optimizers.SGD(.1, momentum=.9, nesterov=True)
    replacement_layers.compile(loss=MSE, optimizer=optimizer)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5, min_lr=.0001, factor=.3, verbose=1)
    early_stop = tf.keras.callbacks.EarlyStopping(patience=15, min_delta=.0001, restore_best_weights=True, verbose=1)
    history = replacement_layers.fit(x=layer_train_gen,
                                   epochs=250,
                                   steps_per_epoch=TRAIN_SIZE // global_batch_size,
                                   validation_data=layer_test_gen,
                                   shuffle=False,
                                   callbacks=[reduce_lr, early_stop],
                                   validation_steps=VALIDATION_SIZE // global_batch_size,
                                   verbose=2)
    target['weights'] = [replacement_layers.layers[1].get_weights(), replacement_layers.layers[3].get_weights()]

training layer conv2_block1_2_conv
  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 520 steps, validate for 104 steps
Epoch 1/250
520/520 - 80s - loss: 0.2173 - val_loss: 0.0531
Epoch 2/250
520/520 - 82s - loss: 0.0420 - val_loss: 0.0327
Epoch 3/250
520/520 - 83s - loss: 0.0301 - val_loss: 0.0281
Epoch 4/250
520/520 - 83s - loss: 0.0267 - val_loss: 0.0257
Epoch 5/250
520/520 - 83s - loss: 0.0252 - val_loss: 0.0242
Epoch 6/250
520/520 - 83s - loss: 0.0240 - val_loss: 0.0232
Epoch 7/250
520/520 - 83s - loss: 0.0233 - val_loss: 0.0235
Epoch 8/250
520/520 - 83s - loss: 0.0228 - val_loss: 0.0232
Epoch 9/250
520/520 - 83s - loss: 0.0224 - val_loss: 0.0221
Epoch 10/250
520/520 - 83s - loss: 0.0212 - val_loss: 0.0203
Epoch 11/250
520/520 - 83s - loss: 0.0205 - val_loss: 0.0201
Epoch 12/250
520/520 - 83s - loss: 0.0202 - val_loss: 0.0198
Epoch 13/250
520/520 - 83s - loss: 0.0200 - val_loss: 0.0198
Epoch 14/250
520/520 - 83s - loss: 0.0199 - val_loss: 0.0198
Epoch 15/250
520/520 - 83s - loss: 0.0196 - val_loss: 0.0195
Epoch 16/250
520/520 - 83s - loss: 0.0195 - val_lo

...

Train for 520 steps, validate for 104 steps
Epoch 1/250
520/520 - 94s - loss: 0.0654 - val_loss: 0.0627
Epoch 2/250
520/520 - 94s - loss: 0.0628 - val_loss: 0.0625
Epoch 3/250
520/520 - 93s - loss: 0.0627 - val_loss: 0.0625
Epoch 4/250
520/520 - 94s - loss: 0.0627 - val_loss: 0.0626
Epoch 5/250
520/520 - 94s - loss: 0.0627 - val_loss: 0.0625
Epoch 6/250
520/520 - 92s - loss: 0.0627 - val_loss: 0.0625
Epoch 7/250

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.030000000447034835.
520/520 - 95s - loss: 0.0627 - val_loss: 0.0625
Epoch 8/250
520/520 - 94s - loss: 0.0627 - val_loss: 0.0625
Epoch 9/250
520/520 - 93s - loss: 0.0627 - val_loss: 0.0625
Epoch 10/250
520/520 - 95s - loss: 0.0627 - val_loss: 0.0625
Epoch 11/250
520/520 - 94s - loss: 0.0626 - val_loss: 0.0625
Epoch 12/250

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.009000000357627868.
520/520 - 93s - loss: 0.0627 - val_loss: 0.0625
Epoch 13/250
520/520 - 94s - loss: 0.0627 - val_loss: 0.0625
Epoch 14/250

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 520 steps, validate for 104 steps
Epoch 1/250
520/520 - 98s - loss: 0.1088 - val_loss: 0.1034
Epoch 2/250
520/520 - 95s - loss: 0.1044 - val_loss: 0.1031
Epoch 3/250
520/520 - 97s - loss: 0.1041 - val_loss: 0.1031
Epoch 4/250
520/520 - 98s - loss: 0.1042 - val_loss: 0.1031
Epoch 5/250
520/520 - 98s - loss: 0.1042 - val_loss: 0.1031
Epoch 6/250
520/520 - 97s - loss: 0.1040 - val_loss: 0.1030
Epoch 7/250
520/520 - 96s - loss: 0.1038 - val_loss: 0.1028
Epoch 8/250
520/520 - 97s - loss: 0.1032 - val_loss: 0.1017
Epoch 9/250
520/520 - 98s - loss: 0.1018 - val_loss: 0.0998
Epoch 10/250
520/520 - 96s - loss: 0.0999 - val_loss: 0.0984
Epoch 11/250
520/520 - 98s - loss: 0.0989 - val_loss: 0.0976
Epoch 12/250
520/520 - 97s - loss: 0.0984 - val_loss: 0.0970
Epoch 13/250
520/520 - 97s - loss: 0.0978 - val_loss: 0.0965
Epoch 14/250
520/520 - 97s - loss: 0.0973 - val_loss: 0.0962
Epoch 15/250
520/520 - 97s - loss: 0.0971 - val_loss: 0.0958
Epoch 16/250
520/520 - 99s - loss: 0.0967 - val_lo

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
for target in targets:
    
    print(f'Replacing Layer {target["name"]}')
    
    tf.keras.backend.clear_session()
    
    model = tf.keras.models.load_model('base_model_cifar10_resnet50.h5')
    
    layers_to_replace = [x[0] for x in target['to_replace']]
    weights_to_replace = [target['to_replace'][0][1], target['to_replace'][-1][1]]
    
    new_model = replace_layer(model, layers_to_replace, lambda x: replac(x))
    new_model.layers[weights_to_replace[0]].set_weights(target['weights'][0])
    new_model.layers[weights_to_replace[1]].set_weights(target['weights'][1])
    new_model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy'])
    target['score'] = new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)
    

Replacing Layer conv2_block1_2_conv
replacing conv2_block1_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv2_block1_2_bn
deleting conv2_block1_2_relu
104/104 [==============================] - 36s 345ms/step - loss: 0.1446 - accuracy: 0.9525
Replacing Layer conv2_block2_2_conv
replacing conv2_block2_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv2_block2_2_bn
deleting conv2_block2_2_relu
104/104 [==============================] - 12s 112ms/step - loss: 0.1430 - accuracy: 0.9530
Replacing Layer conv2_block3_2_conv
replacing conv2_block3_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



...

deleting conv5_block1_2_bn
deleting conv5_block1_2_relu
104/104 [==============================] - 9s 82ms/step - loss: 0.1464 - accuracy: 0.9517
Replacing Layer conv5_block2_2_conv
replacing conv5_block2_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv5_block2_2_bn
deleting conv5_block2_2_relu
104/104 [==============================] - 9s 86ms/step - loss: 0.2146 - accuracy: 0.9348
Replacing Layer conv5_block3_2_conv
replacing conv5_block3_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv5_block3_2_bn
deleting conv5_block3_2_relu
104/104 [==============================] - 9s 84ms/step - loss: 0.1423 - accuracy: 0.9534


In [20]:
for target in targets:
    pprint.pprint(f"name: {target['name']} score: {target['score']}")

'name: conv2_block1_2_conv score: [0.14458354241931096, 0.95252407]'
'name: conv2_block2_2_conv score: [0.1429631541698025, 0.95302486]'
'name: conv2_block3_2_conv score: [0.14124455573395467, 0.9536258]'
'name: conv3_block1_2_conv score: [0.14273115303009176, 0.95272434]'
'name: conv3_block2_2_conv score: [0.1425617459910707, 0.95222354]'
'name: conv3_block3_2_conv score: [0.1430274860288661, 0.95222354]'
'name: conv3_block4_2_conv score: [0.14467338993787193, 0.95292467]'
'name: conv4_block1_2_conv score: [0.14392931879354784, 0.9509215]'
'name: conv4_block2_2_conv score: [0.14221949184026855, 0.9521234]'
'name: conv4_block3_2_conv score: [0.14207436884037003, 0.95232373]'
'name: conv4_block4_2_conv score: [0.1456324967304961, 0.95272434]'
'name: conv4_block5_2_conv score: [0.14340634094193005, 0.95322514]'
'name: conv4_block6_2_conv score: [0.1425402238069532, 0.95252407]'
'name: conv5_block1_2_conv score: [0.14638534492741412, 0.95172274]'
'name: conv5_block2_2_conv score: [0.21462

In [1]:
import pickle

In [2]:
with open('resnet.pickle', 'rb') as f:
    targets = pickle.load(f)

In [11]:
targets[0]['weights'][1][0].shape

(3, 3, 64, 1)

In [32]:
model = tf.keras.models.load_model('base_model_cifar10_resnet50.h5')
for target in targets[::-1]:
    
    if results[1] - target['score'][1] < .01:
    
        print(f'Replacing Layer {target["name"]}')


        layers_to_replace = [x[0] for x in target['to_replace']]
        weights_to_replace = [target['to_replace'][0][1], target['to_replace'][-1][1]]

        model = replace_layer(model, layers_to_replace, lambda x: replac(x))
        model.layers[weights_to_replace[0]].set_weights(target['weights'][0])
        model.layers[weights_to_replace[1]].set_weights(target['weights'][1])



        model.save('resnet_modified.h5')
        tf.keras.backend.clear_session()
        model = tf.keras.models.load_model('resnet_modified.h5')

    
model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy']) 
model.evaluate(test_dataset, steps=VALIDATION_SIZE//global_batch_size)

Replacing Layer conv5_block3_2_conv
replacing conv5_block3_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv5_block3_2_bn
deleting conv5_block3_2_relu


Replacing Layer conv5_block1_2_conv
replacing conv5_block1_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



...

Replacing Layer conv2_block1_2_conv
replacing conv2_block1_2_conv

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



deleting conv2_block1_2_bn
deleting conv2_block1_2_relu


104/104 [==============================] - 12s 116ms/step - loss: 0.1662 - accuracy: 0.9453


[0.1661724245104079, 0.9453125]

In [33]:
model.compile(optimizer=tf.keras.optimizers.SGD(.0001, momentum=.9, nesterov=True), loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(train_dataset,
          epochs=30,
          steps_per_epoch=TRAIN_SIZE//global_batch_size,
          validation_data=test_dataset, 
          validation_steps=VALIDATION_SIZE//global_batch_size)

Train for 520 steps, validate for 104 steps
Epoch 1/30
520/520 [==============================] - 176s 339ms/step - loss: 0.1086 - accuracy: 0.9644 - val_loss: 0.1492 - val_accuracy: 0.9505
Epoch 2/30
520/520 [==============================] - 169s 325ms/step - loss: 0.1031 - accuracy: 0.9661 - val_loss: 0.1447 - val_accuracy: 0.9531
Epoch 3/30
520/520 [==============================] - 168s 323ms/step - loss: 0.1001 - accuracy: 0.9670 - val_loss: 0.1443 - val_accuracy: 0.9503
Epoch 4/30
520/520 [==============================] - 168s 324ms/step - loss: 0.0965 - accuracy: 0.9687 - val_loss: 0.1441 - val_accuracy: 0.9523
Epoch 5/30
520/520 [==============================] - 169s 324ms/step - loss: 0.0951 - accuracy: 0.9694 - val_loss: 0.1446 - val_accuracy: 0.9533
Epoch 6/30
520/520 [==============================] - 169s 326ms/step - loss: 0.0906 - accuracy: 0.9711 - val_loss: 0.1450 - val_accuracy: 0.9522
Epoch 7/30
520/520 [==============================] - 169s 324ms/step - loss: 0.

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [26]:
original = tf.keras.models.load_model('base_model_cifar10_resnet50.h5')

In [26]:
original = tf.keras.models.load_model('base_model_cifar10_resnet50.h5')

In [27]:
original.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________